In [8]:
using Oscar
using Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [3]:
function slicematrix(A)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [4]:
function rowMinors(A,S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [119]:
function convertIncidenceMatrix(A)
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return convert.(Array{Int64, 1}, out)
end
    

convertIncidenceMatrix (generic function with 1 method)

In [116]:
function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end
 

coneListFormat (generic function with 1 method)

In [222]:
function toric_blowup(s, X, v)
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    star = rowMinors(coneList, starIndex)
    rayMatrix = X.RAYS
    
    #d = size(rayMatrix, 1) #length of array describing each ray
    clStar = []
    for t in star
        c = rank(Array(rowMinors(rayMatrix, t))) - 1
        for i in t
            subcone = convert.(Int64, filter(elt -> elt != i, t))
            #println(subcone)
            if rank(Array(rowMinors(rayMatrix, subcone))) == c
                append!(clStar, [subcone])
            end
        end
    end
    clStar = unique(clStar)
                
    n = size(rayMatrix, 1)
    #coneList = filter(t -> !(t in star), coneList)
    coneList  = rowMinors(coneList, filter(i -> !(i in starIndex), range(1, stop=size(coneList, 1))))
    
    if size(s,2) == 1
        newCones = []
        for t in clStar
            if !(s[1] in t)
                append!(newCones, [sort(hcat(t..., s...), dims=2)])
            end
        end
        newCones = convert.(Array{Int64,2}, newCones)
        println(newCones)
        println(convert.(Array{Int64,1}, coneList))
        return fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            append!(t, [n])
        end
    end
    return fulton.NormalToricVariety(INPUT_RAYS = append!(Array(X.RAYS), [v]), INPUT_CONES = append!(coneList, newCones))
end

toric_blowup (generic function with 1 method)

In [ ]:
function toric_blowup(s,X)
    rays = rowMinors(Array(X.RAYS), s)
end

In [14]:
a = Array([1 0; 0 1])
rank(a)

2

In [11]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [223]:
toric_blowup([1], tv, [1 -1])

[[1 2], [1 4]]


LoadError: [91mMethodError: no method matching Array{Int64,1}(::LinearAlgebra.Transpose{Int64,Array{Int64,1}})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  Array{Int64,1}(::AbstractArray{S,N}) where {T, N, S} at array.jl:562[39m
[91m[0m  Array{Int64,1}() where T at boot.jl:425[39m
[91m[0m  Array{Int64,1}([91m::UndefInitializer[39m, [91m::Int64[39m) where T at boot.jl:406[39m
[91m[0m  ...[39m

In [23]:
rank(Polymake.Rational[0 1; 1 0]) - 1

1

In [113]:
coneList = convertIncidenceMatrix(tv.MAXIMAL_CONES)
starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
star = rowMinors(coneList, starIndex)
println(star)
coneList = convert.(Array{Int64, 1}, coneList)
starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
star = rowMinors(coneList, starIndex)
println(star)
rayMatrix = tv.RAYS

1×0 LinearAlgebra.Transpose{Any,Array{Any,1}}
1×0 LinearAlgebra.Transpose{Any,Array{Any,1}}


pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [55]:
s = [0 1 2 3]
size(s, 2)

4

In [99]:
a = Array{Int64, 2}[5 4 3]
print(a)

LoadError: [91mMethodError: [0mCannot `convert` an object of type [92mInt64[39m[0m to an object of type [91mArray{Int64,2}[39m[39m
[91m[0mClosest candidates are:[39m
[91m[0m  convert(::Type{T}, [91m::AbstractArray[39m) where T<:Array at array.jl:554[39m
[91m[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractArray at abstractarray.jl:14[39m
[91m[0m  convert(::Type{T}, [91m::LinearAlgebra.Factorization[39m) where T<:AbstractArray at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/factorization.jl:55[39m
[91m[0m  ...[39m

In [178]:
append!([[1 2], [1 4]], [[2 3], [3 4]])

4-element Array{Array{Int64,2},1}:
 [1 2]
 [1 4]
 [2 3]
 [3 4]

In [190]:
sort(hcat([1, 3]..., [2]...), dims=2)

1×3 Array{Int64,2}:
 1  2  3

In [204]:
convert(Array{Int64, 2}, [[2 3]; [3 4]])

2×2 Array{Int64,2}:
 2  3
 3  4